# Helicóptero de Bancada para disciplina de Controle e Aplicações

## Preparação

In [39]:
# Bibliotecas
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg as la
import ipywidgets as widgets
from ipywidgets import interactive
from scipy.optimize import minimize
from IPython.display import Latex
from tbcontrol.symbolic import routh
from scipy.integrate import solve_ivp
from scipy.signal import place_poles
from scipy.signal import find_peaks

import control as ct
s = ct.tf('s')

from seaborn import color_palette
colors = color_palette()

In [40]:
# Definição dos símbolos
m_c, m_m, a, l, L, K, g = sp.symbols('m_c m_m a l L K g')
epsilon, alpha, theta = sp.symbols('epsilon alpha theta')
d_epsilon, d_alpha, d_theta = sp.symbols('d_epsilon d_alpha d_theta')
dd_epsilon, dd_alpha, dd_theta = sp.symbols('dd_epsilon dd_alpha dd_theta')
V_D, V_E = sp.symbols('V_D V_E')
M_P_x, M_P_y, M_P_z = sp.symbols('M_P_x M_P_y M_P_z')
rho, C_D, U, S, beta = sp.symbols('rho C_D U S beta')
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6') 
u1, u2 = sp.symbols('u1 u2')                              
w1, w2, w3 = sp.symbols('w1 w2 w3')                        
epsilon_linha, alpha_linha, theta_linha = sp.symbols('epsilon_linha alpha_linha theta_linha')
s = sp.symbols('s')

I = sp.eye(6)

In [41]:
# Parametros
params = {
    m_c: 1.565,
    m_m: 0.355,
    l: 0.45,
    L: 0.64,
    K: 0.5839,
    g: 9.81,
    a: 0.275,
    rho: 1.225,
    C_D: 1,
    S: 0.00196,
    beta: 10*np.pi/180,
    U: 5,
    #epsilon_linha: 4*np.pi/180,
    #alpha_linha: 1*np.pi/180,
    theta_linha: 2*np.pi/180
}

In [42]:
# Paleta de cores
cp = {
    'red': (1.0, 0.349, 0.369, 1.0),
    'green': (0.541, 0.788, 0.149, 1.0),
    'blue': (0.098, 0.510, 0.769, 1.0),
    'lred': (1.0, 0.588, 0.6, 1.0),
    'lgreen': (0.722, 0.894, 0.443, 1.0),
    'lblue': (0.369, 0.706, 0.918, 1.0),
    'dred': (0.839,0.0,0.027),
    'dgreen': (0.341,0.494,0.094),
    'dblue': (0.063,0.318,0.475),
    'orange': (1.0, 0.506, 0.227, 1.0),
    'yellow': (1.0, 0.792, 0.227, 1.0),
    'pink': (1.0, 0.349, 0.611, 1.0),
    'purple': (0.416, 0.298, 0.576, 1.0),
    'turquoise': (0.098, 0.761, 0.769, 1.0),
    'brown': (0.576, 0.380, 0.298, 1.0)
}

## Modelagem Matemática

In [43]:
# Vetores unitários
i = sp.Matrix([1, 0, 0])
j = sp.Matrix([0, 1, 0])
k = sp.Matrix([0, 0, 1])

# Matrizes de rotação
R_x = sp.Matrix([
    [1, 0, 0],
    [0, sp.cos(epsilon), -sp.sin(epsilon)],
    [0, sp.sin(epsilon), sp.cos(epsilon)]
])

R_y = sp.Matrix([
    [sp.cos(alpha), 0, sp.sin(alpha)],
    [0, 1, 0],
    [-sp.sin(alpha), 0, sp.cos(alpha)]
])

R_z = sp.Matrix([
    [sp.cos(theta), -sp.sin(theta), 0],
    [sp.sin(theta), sp.cos(theta), 0],
    [0, 0, 1]
])

R = R_x * R_y * R_z

# Vetores de posição
rc = -l * j
r = L * j
rd = r + a * i
re = r - a * i

# Momentos de inércia
Ixx = m_c*l**2 + 2*m_m*L**2
Iyy = 2*m_m*a**2
Izz = m_c*l**2 + 2*m_m*(a**2 + L**2)
J = sp.diag(Ixx, Iyy, Izz)

M_P = M_P_x*i + M_P_y*j + M_P_z*k

# Velocidade e aceleração angular
omega = d_epsilon*i + d_alpha*j + d_theta*k
d_omega = dd_epsilon*i + dd_alpha*j + dd_theta*k

# Pesos
Pc = R * (m_c * g * k)
Pm = R * (m_m * g * k)

# Equilíbrio de forças
Fc = Pc
Fd = Pm + K * V_D * k
Fe = Pm + K * V_E * k

# Arrasto
U_vet = R * (U * (sp.cos(beta) * i + sp.sin(beta) * j))
v_rel = U_vet - omega.cross(r)
modulo_v_rel2 = v_rel.dot(v_rel)
direcao_v_rel = v_rel / sp.sqrt(modulo_v_rel2)
Fa = 0.5 * rho * C_D * modulo_v_rel2 * S * direcao_v_rel

# Soma dos momentos
somatorio_momentos = rc.cross(Fc) + re.cross(Fe) + rd.cross(Fd) + r.cross(Fa) + M_P

# Resolução das EDOs
eqs = somatorio_momentos - (J * d_omega + omega.cross(J * omega))
sol = sp.solve(eqs, (dd_epsilon, dd_alpha, dd_theta), dict=True)[0]

d_d_epsilon = sp.simplify(sol[dd_epsilon])
d_d_alpha = sp.simplify(sol[dd_alpha])
d_d_theta = sp.simplify(sol[dd_theta])

### EDOs do sistema

In [44]:
display(Latex(f'$\\ddot{{\\varepsilon}} = {sp.latex(d_d_epsilon)}$'))
display(Latex(f'$\\ddot{{\\alpha}} = {sp.latex(d_d_alpha)}$'))
display(Latex(f'$\\ddot{{\\theta}} = {sp.latex(d_d_epsilon)}$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [45]:
# Substituições para forma de espaço de estados
d_x4 = d_d_epsilon.subs([
    (epsilon, x1), (alpha, x2), (theta, x3),
    (d_epsilon, x4), (d_alpha, x5), (d_theta, x6),
    (V_D, u1), (V_E, u2),
    (M_P_x, w1), (M_P_y, w2), (M_P_z, w3)
])

d_x5 = d_d_alpha.subs([
    (epsilon, x1), (alpha, x2), (theta, x3),
    (d_epsilon, x4), (d_alpha, x5), (d_theta, x6),
    (V_D, u1), (V_E, u2),
    (M_P_x, w1), (M_P_y, w2), (M_P_z, w3)
])

d_x6 = d_d_theta.subs([
    (epsilon, x1), (alpha, x2), (theta, x3),
    (d_epsilon, x4), (d_alpha, x5), (d_theta, x6),
    (V_D, u1), (V_E, u2),
    (M_P_x, w1), (M_P_y, w2), (M_P_z, w3)
])

# Vetores de estado e entrada
x = sp.Matrix([x1, x2, x3, x4, x5, x6])
u = sp.Matrix([u1, u2])
w = sp.Matrix([w1, w2, w3])

# Equações de estado e saída
f = sp.Matrix([x4, x5, x6, d_x4, d_x5, d_x6])
h = sp.Matrix([x1, x2, x3])

## Ponto de Equilíbrio

In [46]:
# Ponto de equilíbrio
subs_eq = {
    x1: epsilon_linha,
    x2: alpha_linha,
    x3: theta_linha,
    x4: 0,
    x5: 0,
    x6: 0,
    w1: 0,
    w2: 0,
    w3: 0
}

ddX_eq = f.subs(subs_eq)
sol_eq = sp.solve([
    ddX_eq[3].simplify(),  # d_x4 == 0
    ddX_eq[4].simplify()   # d_x5 == 0
], (u1, u2), dict=True)[0]

u1_eq = sp.simplify(sol_eq[u1])
u2_eq = sp.simplify(sol_eq[u2])

In [47]:
display(Latex(f'$u_{{1_{{eq}}}} = {sp.latex(u1_eq)}$'))
display(Latex(f'$u_{{2_{{eq}}}} = {sp.latex(u2_eq)}$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Espaço de Estados Linearizado

In [48]:
# Linearizacao
A = f.jacobian(x)
B = f.jacobian(u)
E = f.jacobian(w)
C = h.jacobian(x)
D = h.jacobian(u)

# Substituições completas com entradas e perturbações de equilíbrio
subs_all_eq = dict(subs_eq)
subs_all_eq[u1] = u1_eq
subs_all_eq[u2] = u2_eq

# Avaliação dos Jacobianos no ponto de equilíbrio
A = A.subs(subs_all_eq)
B = B.subs(subs_all_eq)
C = C.subs(subs_all_eq)
D = D.subs(subs_all_eq)
E = E.subs(subs_all_eq)

In [49]:
display(Latex(f'$A = {sp.latex(A)}$'))
display(Latex(f'$B = {sp.latex(B)}$'))
display(Latex(f'$C = {sp.latex(C)}$'))
display(Latex(f'$D = {sp.latex(D)}$'))
display(Latex(f'$E = {sp.latex(E)}$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [50]:
# Aproximação para pequenos ângulos: sin(a) = a, cos(a)=1, sin(a+b)=a+b e cos(a+b)=1-a*b
small_angle_subs = {
    sp.sin(epsilon_linha): epsilon_linha,
    sp.cos(epsilon_linha): 1,
    sp.sin(alpha_linha): alpha_linha,
    sp.cos(alpha_linha): 1,
    sp.sin(beta + theta_linha): beta + theta_linha,
    sp.cos(beta + theta_linha): 1 - beta * theta_linha
}

A = sp.simplify(A.subs(small_angle_subs))
B = sp.simplify(B.subs(small_angle_subs))
C = sp.simplify(C.subs(small_angle_subs))
D = sp.simplify(D.subs(small_angle_subs))
E= sp.simplify(E.subs(small_angle_subs))

In [51]:
display(Latex(f'$A = {sp.latex(A)}$'))
display(Latex(f'$B = {sp.latex(B)}$'))
display(Latex(f'$C = {sp.latex(C)}$'))
display(Latex(f'$D = {sp.latex(D)}$'))
display(Latex(f'$E = {sp.latex(E)}$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [52]:
# Substitituicao numerica
A_num = A.subs(params).evalf()
B_num = B.subs(params).evalf()
C_num = C.subs(params).evalf()
D_num = D.subs(params).evalf()
E_num = E.subs(params).evalf()

### Ordens do sistema

In [53]:
n = A_num.shape[0]
r = B_num.shape[1]
m = C_num.shape[0]
l = E_num.shape[1]

print("n - Ordem do sistema:", n)
print("r - Ordem da entrada:", r)
print("m - Ordem da saída:", m)
print("l - Ordem da perturbação:", l)

n - Ordem do sistema: 6
r - Ordem da entrada: 2
m - Ordem da saída: 3
l - Ordem da perturbação: 3


## Análise de Estabilidade

In [54]:
p = sp.simplify(A_num.charpoly('s'))
p

PurePoly(1.0*s**6 + (0.0039964518169993*alpha_linha**2 - 0.00168429111074122*alpha_linha*epsilon_linha + 0.000177459698979425*epsilon_linha**2 + 0.0114347965906679)*s**5 + (1.48555213437681e-5*alpha_linha**2 - 0.0314199236077218*alpha_linha*epsilon_linha + 6.59649225503965e-7*epsilon_linha**2 - 4.03309783891985*epsilon_linha - 0.0126719149803521)*s**4 + (-3.3881317890172e-21*alpha_linha**2 - 0.00011164805242729*alpha_linha*epsilon_linha - 2.53866384789675e-5*epsilon_linha**2 - 0.0298013688584842*epsilon_linha - 7.35196813385181e-5)*s**3 + (5.29395592033938e-23*alpha_linha**2*epsilon_linha**2 + 5.42101086242752e-20*alpha_linha**2*epsilon_linha - 7.94093388050907e-23*alpha_linha**2 + 9.92616735063633e-24*alpha_linha*epsilon_linha**3 - 1.01643953670516e-20*alpha_linha*epsilon_linha**2 + 0.000191064735730922*alpha_linha*epsilon_linha + 0.0245301790406917*epsilon_linha + 4.0261793548826e-5)*s**2, s, domain='RR[alpha_linha,epsilon_linha]')

In [55]:
RHm = routh(p)
RHm

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [56]:
RHm.subs({sp.nan: 0, alpha_linha: 2*np.pi/180, epsilon_linha:-0.3275*np.pi/180})

Matrix([
[                 1.0,    0.010387371086653, -9.99896863739711e-5, 0],
[  0.0114400080012061,  9.68449984174697e-5,                    0, 0],
[ 0.00192190511781283, -9.99896863739711e-5,                    0, 0],
[0.000692026728022353,                    0,                    0, 0],
[ -9.9989686373971e-5,                    0,                    0, 0],
[                   0,                    0,                    0, 0],
[                   0,                    0,                    0, 0]])

In [57]:
# Adicionando os valores de alpha_linha e epsilon_linha que melhor estabilizam o sistema aos parametros
params["alpha_linha"] = 2*np.pi/180
params["epsilon_linha"] = -0.3275*np.pi/180

A_num = A.subs(params).evalf()
B_num = B.subs(params).evalf()
C_num = C.subs(params).evalf()
D_num = D.subs(params).evalf()
E_num = E.subs(params).evalf()
p = sp.simplify(A_num.charpoly('s'))

## Função de Transferência

### Espaço de Estado

In [58]:
heli = ct.ss(np.array(A_num), np.array(B_num), np.array(C_num), np.array(D_num))
heli

StateSpace(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-1.64396757e-02,  1.09368010e-01,  5.15082868e-05,
        -4.05080743e-03,  0.00000000e+00, -1.44316150e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.70670158e+00,  6.08221769e-03,
        -1.32600676e-04,  0.00000000e+00, -7.38920057e-03]]), array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.61490616,  0.61490616],
       [-2.99052497,  2.99052497],
       [ 0.        ,  0.        ]]), array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 

### FT e Polos

In [59]:
G = ct.ss2tf(heli)
G

ControlMIMONotImplemented: Not implemented for MIMO systems without slycot.

In [ ]:
poles_MA = ct.poles(heli)
display(Latex(f'${sp.latex(sp.Matrix(poles_MA))}$'))

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

# Polos: como 'x'
ax.scatter(poles_MA.real, poles_MA.imag, marker='x', color='red', label='Polos MA')


ax.set_xlabel('Parte Real')
ax.set_ylabel('Parte Imaginária')
ax.set_title('Mapa de Polos - Malha Aberta')
ax.grid(False)
ax.axhline(0, color='lightgrey', linestyle='-', linewidth=1.5)
ax.axvline(0, color='lightgrey', linestyle='-', linewidth=1.5)
ax.legend()
plt.show()

### Resposta em frequência

In [ ]:
# Dicionário de saídas
saidas = [
    (0, 'x₁', cp['red']),
    (1, 'x₂', cp['blue']),
    (2, 'x₃', cp['green']),
]

# Frequência comum
omega = None  # será definida na 1ª chamada

# Função para plotar para uma entrada
def plot_bode_entrada(j, titulo_entrada):
    global omega

    fig_mag, ax_mag = plt.subplots(figsize=(10, 4))
    fig_phase, ax_phase = plt.subplots(figsize=(10, 4))

    for i, saida_label, color in saidas:
        mag, phase, omega = ct.bode_plot(G[i, j], dB=True, Hz=True, omega=omega, plot=False)
        freq = omega / (2 * np.pi)

        ax_mag.semilogx(freq, 20 * np.log10(mag), label=f'{titulo_entrada} → {saida_label}', color=color)
        ax_phase.semilogx(freq, phase * 180 / np.pi, label=f'{titulo_entrada} → {saida_label}', color=color)

    # Magnitude
    ax_mag.set_title(f'Magnitude (entrada {titulo_entrada})')
    ax_mag.set_xlabel('Frequência (Hz)')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True, which='both')
    ax_mag.legend()

    # Fase
    ax_phase.set_title(f'Fase (entrada {titulo_entrada})')
    ax_phase.set_xlabel('Frequência (Hz)')
    ax_phase.set_ylabel('Fase (°)')
    ax_phase.grid(True, which='both')
    ax_phase.legend()

    plt.tight_layout()
    plt.show()

# Plot para u1 (coluna j=0) e u2 (coluna j=1)
plot_bode_entrada(0, 'u₁')
plot_bode_entrada(1, 'u₂')


## Controlabilidade e Observabilidade

In [ ]:
# Matriz de controlabilidade
Q = ct.ctrb(A_num, B_num)
sp.Matrix(Q)

In [ ]:
print("O sistema é controlável?", sp.Matrix(Q).rank()==n)

In [ ]:
# Matriz de observabilidade
N = ct.obsv(A_num, C_num)
sp.Matrix(N)

In [ ]:
print("O sistema é observável?", sp.Matrix(N).rank()==n)

## Regulador linear-quadrático (LQR)

In [ ]:
omega_f = (poles_MA[0].real**2 + poles_MA[0].imag**2)**0.5

In [ ]:
# Matriz de ponderação Q - penaliza desvios dos estados
Q = np.diag([5, 20, 1, 5/(omega_f**2), 20/(omega_f**2), 1/(omega_f**2)])

# Matriz de ponderação R - penaliza esforço de controle
R = np.diag([10**4, 10**4])

# Síntese do regulador LQR
K_LQR, P_LQR, poles_LQR_MF = ct.lqr(np.array(A_num).astype(np.float64), np.array(B_num).astype(np.float64), Q, R)

# Sistema em malha fechada
A_LQR = np.array(A_num).astype(np.float64) - np.array(B_num).astype(np.float64) @ K_LQR
N_ff = -np.linalg.inv(np.array(C_num).astype(np.float64) @ np.linalg.inv(A_LQR) @ np.array(B_num).astype(np.float64))
heli_LQR_MF = ct.ss(A_LQR_MF, E_num,-K_LQR, sp.zeros(K_LQR.shape[0],E_num.shape[1]))
G_LQR_MF = ct.ss2tf(heli_LQR_MF)

In [ ]:
display(Latex(f'$Q = {sp.latex(sp.Matrix(Q))}$'))
display(Latex(f'$R = {sp.latex(sp.Matrix(R))}$'))
display(Latex(f'$K = {sp.latex(sp.Matrix(K_LQR))}$'))

### Comparação de Polos

In [ ]:
poles_LQR_MF = ct.poles(heli_LQR_MF)

# Garantir mesmo tamanho com preenchimento de NaN
max_poles = max(len(poles_MA), len(poles_LQR_MF))

# Criar DataFrame
df_poles = pd.DataFrame({
    'Polos MA': np.pad(poles_MA, (0, max_poles - len(poles_MA)), constant_values=np.nan),
    'Polos MF': np.pad(poles_LQR_MF, (0, max_poles - len(poles_LQR_MF)), constant_values=np.nan)
})

# Exibir tabelas
display(df_poles)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

# Polos: como 'x'
ax.scatter(poles_MA.real, poles_MA.imag, marker='x', color='red', label='Polos MA')
ax.scatter(poles_LQR_MF.real, poles_LQR_MF.imag, marker='x', color='blue', label='Polos MF - LQR')


ax.set_xlabel('Parte Real')
ax.set_ylabel('Parte Imaginária')
ax.set_title('Mapa de Polos - Malha Aberta e Malha Fechada')
ax.grid(False)
ax.axhline(0, color='lightgrey', linestyle='-', linewidth=1.5)
ax.axvline(0, color='lightgrey', linestyle='-', linewidth=1.5)
ax.legend()
plt.show()

### Resposta em frequência

In [ ]:
# Dicionário de saídas
saidas = [
    (0, 'u₁', cp['red']),
    (1, 'u₂', cp['blue']),
]

# Frequência comum
omega = None  # será definida na 1ª chamada

# Função para plotar para uma entrada
def plot_bode_entrada(j, titulo_entrada):
    global omega

    fig_mag, ax_mag = plt.subplots(figsize=(10, 4))
    fig_phase, ax_phase = plt.subplots(figsize=(10, 4))

    for i, saida_label, color in saidas:
        mag, phase, omega = ct.bode_plot(G_LQR_MF[i, j], dB=True, Hz=True, omega=omega, plot=False)
        freq = omega / (2 * np.pi)

        ax_mag.semilogx(freq, 20 * np.log10(mag), label=f'{titulo_entrada} → {saida_label}', color=color)
        ax_phase.semilogx(freq, phase * 180 / np.pi, label=f'{titulo_entrada} → {saida_label}', color=color)

    # Magnitude
    ax_mag.set_title(f'Magnitude (perturbação {titulo_entrada})')
    ax_mag.set_xlabel('Frequência (Hz)')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True, which='both')
    ax_mag.legend()

    # Fase
    ax_phase.set_title(f'Fase (perturbação {titulo_entrada})')
    ax_phase.set_xlabel('Frequência (Hz)')
    ax_phase.set_ylabel('Fase (°)')
    ax_phase.grid(True, which='both')
    ax_phase.legend()

    plt.tight_layout()
    plt.show()

# Plot para u1 (coluna j=0) e u2 (coluna j=1)
plot_bode_entrada(0, 'w₁')
plot_bode_entrada(1, 'w₂')
plot_bode_entrada(2, 'w3')

### Simulação de resposta livre do sistema controlado

In [ ]:
ts = np.linspace(0, 50, 1001)
w = np.zeros((E_num.shape[1], len(ts)))
x0 = [2*np.pi/180, np.pi/180, 7*np.pi/180, 0, 0, 0] ## ALTERÁVEL
t_livre, y_livre, x_livre = ct.forced_response(heli_LQR_MF, T=ts, X0=x0, return_x=True)

plt.figure(figsize=(12, 5))
for i in range(x.shape[0]):
    plt.plot(t_livre, x_livre[i, :], label=f"$x_{i+1}(t)$")
plt.title(f"Resposta Livre - Estados (LQR)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
for i in range(y_livre.shape[0]):
    plt.plot(t_livre, y_livre[i, :], label=f"$u_{i+1}(t)$")
plt.title(f"Resposta Livre - Entradas de controle (LQR)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

### Simulação de resposta a perturbação de impulso em sistema controlado

In [ ]:
ts = np.linspace(0, 60, 1001)
x0 = np.zeros(6)

w_impulse = np.zeros((E_num.shape[1], len(ts)))
w_impulse[0, 0] = 1 / (ts[1] - ts[0])  # impulso aproximado como valor alto no primeiro passo

# Simulação com perturbação
t_imp, y_imp, x_imp = ct.forced_response(heli_LQR_MF, T=ts, U=w_impulse, X0=x0, return_x=True)

# Plot dos estados
plt.figure(figsize=(12, 5))
for i in range(x_imp.shape[0]):
    plt.plot(t_imp, x_imp[i, :], label=f"$x_{i+1}(t)$")
plt.title("Resposta a uma perturbação tipo impulso - Estados (LQR)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Plot das entradas de controle
plt.figure(figsize=(10, 4))
for i in range(y_imp.shape[0]):
    plt.plot(t_imp, y_imp[i, :], label=f"$u_{i+1}(t)$")
plt.title("Resposta a uma perturbação tipo impulso - Entradas de controle (LQR)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Regulador por alocação de polos

Os polos serão alocados em:
* $s_1 = - 0.8$,
* $s_2 = - 0.12$ 
* $s_3 = - 0.4 + 0.2j$,
* $s_4 = - 0.4 - 0.2j$
* $s_5 = - 0.1 + 0.1j$,
* $s_6 = - 0.1 - 0.1j$.

In [ ]:
# Polinômio característico do sistema
p_aloc = sp.PurePoly((s+0.8)*(s+0.12)*((s + 0.4)**2 + 0.2**2)*((s + 0.1)**2 + 0.1**2), s)
p_aloc

In [ ]:
# Polos alocados do sistema em MF
poles_aloc_MF = np.roots(p_aloc.all_coeffs())

# Síntese do regulador por alocação de polos
K_aloc = place_poles(np.array(A_num).astype(np.float64), np.array(B_num).astype(np.float64), poles_aloc_MF).gain_matrix

# Sistema em malha fechada
A_aloc_MF = np.array(A_num).astype(np.float64) - np.array(B_num).astype(np.float64) @ K_aloc
heli_aloc_MF = ct.ss(A_aloc_MF, E_num,-K_aloc, sp.zeros(K_aloc.shape[0],E_num.shape[1]))
G_aloc_MF = ct.ss2tf(heli_aloc_MF)

In [ ]:
display(Latex(f'$K = {sp.latex(sp.Matrix(K_aloc))}$'))

### Comparação de Polos

In [ ]:
poles_aloc_MF = ct.poles(heli_aloc_MF)

# Garantir mesmo tamanho com preenchimento de NaN
max_poles = max(len(poles_MA), len(poles_aloc_MF))

# Criar DataFrame
df_poles = pd.DataFrame({
    'Polos MA': np.pad(poles_MA, (0, max_poles - len(poles_MA)), constant_values=np.nan),
    'Polos MF': np.pad(poles_aloc_MF, (0, max_poles - len(poles_aloc_MF)), constant_values=np.nan)
})

# Exibir tabelas
display(df_poles)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

# Polos: como 'x'
ax.scatter(poles_MA.real, poles_MA.imag, marker='x', color='red', label='Polos MA')
ax.scatter(poles_aloc_MF.real, poles_aloc_MF.imag, marker='x', color='blue', label='Polos MF - Alocação')


ax.set_xlabel('Parte Real')
ax.set_ylabel('Parte Imaginária')
ax.set_title('Mapa de Polos - Malha Aberta e Malha Fechada')
ax.grid(False)
ax.axhline(0, color='lightgrey', linestyle='-', linewidth=1.5)
ax.axvline(0, color='lightgrey', linestyle='-', linewidth=1.5)
ax.legend()
plt.show()

### Resposta em frequência

In [ ]:
# Dicionário de saídas
saidas = [
    (0, 'u₁', cp['red']),
    (1, 'u₂', cp['blue']),
]

# Frequência comum
omega = None  # será definida na 1ª chamada

# Função para plotar para uma entrada
def plot_bode_entrada(j, titulo_entrada):
    global omega

    fig_mag, ax_mag = plt.subplots(figsize=(10, 4))
    fig_phase, ax_phase = plt.subplots(figsize=(10, 4))

    for i, saida_label, color in saidas:
        mag, phase, omega = ct.bode_plot(G_aloc_MF[i, j], dB=True, Hz=True, omega=omega, plot=False)
        freq = omega / (2 * np.pi)

        ax_mag.semilogx(freq, 20 * np.log10(mag), label=f'{titulo_entrada} → {saida_label}', color=color)
        ax_phase.semilogx(freq, phase * 180 / np.pi, label=f'{titulo_entrada} → {saida_label}', color=color)

    # Magnitude
    ax_mag.set_title(f'Magnitude (perturbação {titulo_entrada})')
    ax_mag.set_xlabel('Frequência (Hz)')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True, which='both')
    ax_mag.legend()

    # Fase
    ax_phase.set_title(f'Fase (perturbação {titulo_entrada})')
    ax_phase.set_xlabel('Frequência (Hz)')
    ax_phase.set_ylabel('Fase (°)')
    ax_phase.grid(True, which='both')
    ax_phase.legend()

    plt.tight_layout()
    plt.show()

# Plot para u1 (coluna j=0) e u2 (coluna j=1)
plot_bode_entrada(0, 'w₁')
plot_bode_entrada(1, 'w₂')
plot_bode_entrada(2, 'w3')

### Simulação de resposta livre do sistema controlado

In [ ]:
ts = np.linspace(0, 60, 1001)
w = np.zeros((E_num.shape[1], len(ts)))
x0 = [2*np.pi/180, np.pi/180, 7*np.pi/180, 0, 0, 0] ## ALTERÁVEL
t_livre, y_livre, x_livre = ct.forced_response(heli_aloc_MF, T=ts, X0=x0, return_x=True)

plt.figure(figsize=(12, 5))
for i in range(x.shape[0]):
    plt.plot(t_livre, x_livre[i, :], label=f"$x_{i+1}(t)$")
plt.title(f"Resposta Livre - Estados (Alocação)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
for i in range(y_livre.shape[0]):
    plt.plot(t_livre, y_livre[i, :], label=f"$u_{i+1}(t)$")
plt.title(f"Resposta Livre - Entradas de controle (Alocação)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

### Simulação de resposta a perturbação de impulso em sistema controlado

In [ ]:
ts = np.linspace(0, 60, 1001)
x0 = np.zeros(6)

w_impulse = np.zeros((E_num.shape[1], len(ts)))
w_impulse[0, 0] = 1 / (ts[1] - ts[0])  # impulso aproximado como valor alto no primeiro passo

# Simulação com perturbação
t_imp, y_imp, x_imp = ct.forced_response(heli_aloc_MF, T=ts, U=w_impulse, X0=x0, return_x=True)

# Plot dos estados
plt.figure(figsize=(12, 5))
for i in range(x_imp.shape[0]):
    plt.plot(t_imp, x_imp[i, :], label=f"$x_{i+1}(t)$")
plt.title("Resposta a uma perturbação tipo impulso - Estados (Alocação)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Plot das entradas de controle
plt.figure(figsize=(10, 4))
for i in range(y_imp.shape[0]):
    plt.plot(t_imp, y_imp[i, :], label=f"$u_{i+1}(t)$")
plt.title("Resposta a uma perturbação tipo impulso - Entradas de controle (Alocação)")
plt.xlabel("Tempo [s]")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Observador de Estados de Ordem Reduzida

Como o controlador LQR (por alocação de polos) é mais adequado

In [ ]:
K = K_LQR
poles_cont = poles_LQR_MF
A_cont = A_LQR_MF

In [ ]:
C_num.rank() == m

In [ ]:
# Matrizes auxiliares da transformação de variáveis
V = la.null_space(np.array(C_num).astype(np.float64), rcond=1e-10).T
T = np.block([[np.array(C_num).astype(np.float64)], [V]])
Ti = la.inv(T)
M = Ti[:, 0:m]
N = Ti[:, m:]
A11 = np.array(C_num).astype(np.float64) @ np.array(A_num).astype(np.float64) @ M
A12 = np.array(C_num).astype(np.float64) @ np.array(A_num).astype(np.float64) @ N
A21 = V @ np.array(A_num).astype(np.float64) @ M
A22 = V @ np.array(A_num).astype(np.float64) @ N
B1 = np.array(C_num).astype(np.float64) @ np.array(B_num).astype(np.float64)
B2 = V @ np.array(B_num).astype(np.float64)

In [ ]:
# Teste de observabilidade
np.linalg.matrix_rank(ct.obsv(A22, A12)) == A22.shape[0]

Os polos desejados pelo observador são: ! A DETERMINAR
* $s_1 = - 3 - 2j$,
* $s_2 = - 3 + 2j$ 
* $s_3 = - 4$.

In [ ]:
# Polos do observador
poles_obs = [-3-2j, -3+2j, -4]

# Alocação de polos do observador
J = (place_poles(A22.T, A12.T, poles_obs).gain_matrix).T

# Matrizes F, G, H e S

F = A22 - J @ A12
G_obs = A21  - J @ A11 + F @ J
H = B2 - J @ B1
S = M + N @ J

Colocando o sistema na forma de espaço de estados:

\begin{align*}
    \frac{\mathrm{d}}{\mathrm{d}t}
    \begin{bmatrix}
        x(t) \\ z(t)
    \end{bmatrix}
    = 
    \begin{bmatrix}
        A & 0 \\ GC & F
    \end{bmatrix}
    \begin{bmatrix}
        x(t) \\ z(t)
    \end{bmatrix}
    +
    \begin{bmatrix}
        B \\ H
    \end{bmatrix}
    u(t)
\end{align*}

In [ ]:
A_obs = np.block([[A_num, np.zeros((n, n-m))], [G_obs @ C_num, F]])
B_obs = np.block([[B_num], [H]])
E_obs = np.block([[E_num], [np.zeros((n-m, l))]])
C_obs = np.block([[C_num, np.zeros((m, n-m))]])

\begin{align*}
    \Rightarrow\frac{\mathrm{d}}{\mathrm{d}t}
    \begin{bmatrix}
        x(t) \\ z(t)
    \end{bmatrix}
    = 
   A_{obs}
    \begin{bmatrix}
        x(t) \\ z(t)
    \end{bmatrix}
    +
    B_{obs}
    u(t)
    + E_{obs} w(t)
\end{align*}

\begin{align*}
    y(t)=C_{obs} \begin{bmatrix}
        x(t) \\ z(t)
    \end{bmatrix}
\end{align*}

In [ ]:
display(Latex(f'$A_{{obs}} = {sp.latex(sp.Matrix(A_obs))}$'))
display(Latex(f'$B_{{obs}} = {sp.latex(sp.Matrix(B_obs))}$'))
display(Latex(f'$E_{{obs}} = {sp.latex(sp.Matrix(E_obs))}$'))
display(Latex(f'$C_{{obs}} = {sp.latex(sp.Matrix(C_obs))}$'))

### Simulação de teste

Adotando a lei de controle:
\begin{align*}
    u(t) = - K \hat x(t) = - K \begin{bmatrix}
        SC & N
    \end{bmatrix}
    \begin{bmatrix}
        x(t) \\ z(t)
    \end{bmatrix}
\end{align*}
pode-se estabelecer a dinâmica do sistema aumentado em malha-fechada:

In [ ]:
# Sistema com observador
A_obs_cont = A_obs - B_obs @ K @ np.block([S @ C_num, N])
K_obs_cont = K @ np.block([S @ C_num, N])
 
heli_obs_cont = ct.ss(A_obs_cont, E_obs, -K_obs_cont, sp.zeros(2,3)) # mapeia perturbação w(t) para entrada u(t)
G_obs_cont = ct.ss2tf(heli_obs_cont)

In [ ]:
# Dicionário de saídas
saidas = [
    (0, 'u₁', cp['red']),
    (1, 'u₂', cp['blue']),
]

# Frequência comum
omega = None  # será definida na 1ª chamada

# Função para plotar para uma entrada
def plot_bode_entrada(j, titulo_entrada):
    global omega

    fig_mag, ax_mag = plt.subplots(figsize=(10, 4))
    fig_phase, ax_phase = plt.subplots(figsize=(10, 4))

    for i, saida_label, color in saidas:
        mag, phase, omega = ct.bode_plot(G_obs_cont[i, j], dB=True, Hz=True, omega=omega, plot=False)
        freq = omega / (2 * np.pi)

        ax_mag.semilogx(freq, 20 * np.log10(mag), label=f'{titulo_entrada} → {saida_label}', color=color)
        ax_phase.semilogx(freq, phase * 180 / np.pi, label=f'{titulo_entrada} → {saida_label}', color=color)

    # Magnitude
    ax_mag.set_title(f'Magnitude (perturbação {titulo_entrada})')
    ax_mag.set_xlabel('Frequência (Hz)')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True, which='both')
    ax_mag.legend()

    # Fase
    ax_phase.set_title(f'Fase (perturbação {titulo_entrada})')
    ax_phase.set_xlabel('Frequência (Hz)')
    ax_phase.set_ylabel('Fase (°)')
    ax_phase.grid(True, which='both')
    ax_phase.legend()

    plt.tight_layout()
    plt.show()

# Plot para u1 (coluna j=0) e u2 (coluna j=1)
plot_bode_entrada(0, 'w₁')
plot_bode_entrada(1, 'w₂')
plot_bode_entrada(2, 'w3')

In [ ]:
# Condições iniciais coerentes com ângulos e velocidades pequenas
x0_real = np.array([0.2*np.pi/180, np.pi/180, 7*np.pi/180, 0, 0, 0])  # rad, rad/s
z0 = np.zeros(n - m)  # z(t=0) = 0

# Estado inicial total [x(0); z(0)]
x0z0 = np.concatenate([x0_real, z0])
# Tempo e entrada
t = np.linspace(0, 60, 1000)
w = np.zeros((3, len(t)))  # Matriz com 3 entradas, 1000 amostras

# Simulação
t_obs_cont, y_obs_cont, x_obs_cont = ct.forced_response(heli_obs_cont, T=t, U=w, X0=x0z0, return_x=True)
x_hat_t = np.array(S @ C_num @ x_obs_cont[0:n, :] + N @ x_obs_cont[n:, :]).astype(np.float64)  # estimativa dos estados reais

plt.figure(figsize=(12, 6))

for i in range(n): 
    cor = cp[['lred', 'lgreen', 'lblue', 'dred', 'dgreen', 'dblue'][i]]
    plt.plot(t_obs_cont, x_obs_cont[i, :], label=f'$x_{{{i+1}}}$ real', color=cor)
    plt.plot(t_obs_cont, x_hat_t[i, :].flatten(), '--', label=f'$\\hat{{x}}_{{{i+1}}}$ estimado', color=cor)

plt.xlabel('Tempo [s]')
plt.ylabel('Estados [rad / rad/s]')
plt.title('Estimativas de Estados: Reais vs Estimados')
plt.legend(ncol=2)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
erro_estimacao = np.linalg.norm(x_hat_t - x_obs_cont[0:n, :], axis=0)

import matplotlib.pyplot as plt
plt.semilogy(t_obs_cont, erro_estimacao)
plt.xlabel('Tempo')
plt.ylabel('Erro de estimação (norma)')
plt.grid()
plt.show()

## Cenários de Simulação

In [ ]:
def cenario1():
    x0 = np.zeros(n)

    def w_func(t):
        # Pulso de 0.015 N·m em M_Pz durante 0.1s (de t = 1.0 a t = 1.1)
        if 1.0 <= t <= 1.1:
            return np.array([0.0, 0.0, 0.015])
        else:
            return np.zeros(3)

    def x_ref_func(t):
        return np.zeros(6)

    return {'x0': x0, 'w_func': w_func, 'x_ref_func': x_ref_func}


In [ ]:
def cenario2():
    x0 = np.zeros(n)

    def w_func(t):
        # Perturbação constante em M_Px
        return np.array([0.002, 0.0, 0.0])

    def x_ref_func(t):
        # ε = 0.1 * sin(0.5t), α = 0.05 * cos(0.5t)
        x_ref = np.zeros(6)
        x_ref[0] = 0.1 * np.sin(0.5 * t)   # ε
        x_ref[2] = 0.05 * np.cos(0.5 * t)  # α
        return x_ref

    return {'x0': x0, 'w_func': w_func, 'x_ref_func': x_ref_func}

## Simulações do sistema com regulador e observador

In [ ]:
def simular_cenario(cenario, T, title):
    # Tempo de simulação
    t = np.linspace(0, T, 1000)
    # Parâmetros do cenário
    x0 = cenario()['x0']
    w = np.array([cenario()['w_func'](ti) for ti in t]).T  # shape (3, len(t))

    # Condição inicial estendida [x0; z0]
    z0 = np.zeros(n - m)
    x0_ext = np.concatenate((x0, z0))

    # Simulação
    t_out, y_out, x_out = ct.forced_response(heli_obs_cont, T=t, U=w, X0=x0_ext, return_x=True)
    x_hat_t = np.array(S @ C_num @ x_out[0:n, :] + N @ x_out[n:, :]).astype(np.float64)  # estimativa dos estados reais

    plt.figure(figsize=(12, 6))
    for i in range(n):
        cor = cp[['lred', 'lgreen', 'lblue', 'dred', 'dgreen', 'dblue'][i]]
        plt.plot(t_out, x_out[i, :], label=f'$x_{{{i+1}}}$ real', color = cor)
        plt.plot(t_out, x_hat_t[i, :].flatten(), '--', label=f'$\\hat{{x}}_{{{i+1}}}$ estimado', color = cor)

    plt.xlabel('Tempo [s]')
    plt.ylabel('Estados [rad / rad/s]')
    plt.title(title)
    plt.legend(ncol=2)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
simular_cenario(cenario1, T=60, title ='Cenário 1')

In [ ]:
simular_cenario(cenario2, T=60, title ='Cenário 2')

## Síntese de pré-alimentação

### Seguidor LQ

In [ ]:
Q_LQ = np.diag([10, 10, 10, 1, 1, 1])

R_LQ = np.diag([10**0, 10**0])

# Síntese do regulador LQR
K_LQ, P_LQ, poles_LQ = ct.lqr(np.array(A_num).astype(np.float64), np.array(B_num).astype(np.float64), Q_LQ, R_LQ)
A_LQ = np.array(A_num).astype(np.float64) - np.array(B_num).astype(np.float64) @ K_LQ

### Sistema com pré-alimentação

In [ ]:
# Definindo referências de rastreamento
ts = np.linspace(0, 40, 1001)
# Referência: curva suave com variação em epsilon e alpha
x_ref = np.zeros((6, len(ts)))
x_ref[0, :] = 0.1 * np.sin(0.5 * ts)     # ε (guinada)
x_ref[2, :] = 0.05 * np.cos(0.5 * ts)    # α (inclinação lateral)


# Custo terminal backward
η0 = P_LQ @ x_ref[:, -1].reshape(-1, 1)

# Sistema auxiliar (tempo reverso)
η_ss = ct.ss(-A_LQ, -Q_LQ, np.eye(n), np.zeros_like(Q_LQ))
ts_rev, η = ct.forced_response(η_ss, T=ts[::-1], U=x_ref[:, ::-1], X0=η0)
η = η[:, ::-1]  # reverter para frente no tempo

# Controle final via convolução com matriz de controle
B_aux = np.array(B_num).astype(float) @ np.linalg.inv(R_LQ) @ np.array(B_num).astype(float).T
D_aux = np.zeros((K_LQ.shape[0], B_aux.shape[1]))
sys_c_ss = ct.ss(A_LQ, B_aux, -K_LQ, D_aux)
ts_d, u_lq, x_lq = ct.forced_response(sys_c_ss, T=ts, U=η, return_x=True)

In [ ]:
fig, axs = plt.subplots(7, 1, figsize=(8, 12), sharex=True)

# Estados de posição angular (rad para graus)
axs[0].plot(ts, x_ref[0, :] * 180/np.pi, 'b--', label=r'Ref $\varepsilon$')
axs[0].plot(ts, x_lq[0, :] * 180/np.pi, 'b', label=r'$\varepsilon$')
axs[0].set_ylabel('ε (°)')
axs[0].legend()

axs[1].plot(ts, x_ref[1, :] * 180/np.pi, 'r--', label=r'Ref $\alpha$')
axs[1].plot(ts, x_lq[1, :] * 180/np.pi, 'r', label=r'$\alpha$')
axs[1].set_ylabel('α (°)')
axs[1].legend()

axs[2].plot(ts, x_ref[2, :] * 180/np.pi, 'g--', label=r'Ref $\theta$')
axs[2].plot(ts, x_lq[2, :] * 180/np.pi, 'g', label=r'$\theta$')
axs[2].set_ylabel('θ (°)')
axs[2].legend()

# Estados de velocidade angular (rad/s)
axs[3].plot(ts, x_ref[3, :], 'b--', label=r'Ref $\dot{\varepsilon}$')
axs[3].plot(ts, x_lq[3, :], 'b', label=r'$\dot{\varepsilon}$')
axs[3].set_ylabel('ε̇ (rad/s)')
axs[3].legend()

axs[4].plot(ts, x_ref[4, :], 'r--', label=r'Ref $\dot{\alpha}$')
axs[4].plot(ts, x_lq[4, :], 'r', label=r'$\dot{\alpha}$')
axs[4].set_ylabel('α̇ (rad/s)')
axs[4].legend()

axs[5].plot(ts, x_ref[5, :], 'g--', label=r'Ref $\dot{\theta}$')
axs[5].plot(ts, x_lq[5, :], 'g', label=r'$\dot{\theta}$')
axs[5].set_ylabel('θ̇ (rad/s)')
axs[5].legend()

# Entradas de controle
axs[6].plot(ts, u_lq[0, :], 'k', label=r'$V_{D}$')
axs[6].plot(ts, u_lq[1, :], 'gray', label=r'$V_{E}$')
axs[6].set_ylabel('V (V)')
axs[6].set_xlabel('Tempo (s)')
axs[6].legend()

plt.tight_layout()
plt.show()

## Simulações em Malha Fechada

In [ ]:
def simular_cenario_com_pre_alimentacao(cenario, T, title):
    # Tempo de simulação: cria um vetor de tempo de 0 até T com 1001 pontos
    t = np.linspace(0, T, 1001)
    
    # Obtém a condição inicial x0 do cenário
    x0 = cenario()['x0']
    
    # Gera as perturbações w(t) ao longo do tempo, shape resultante (3, len(t))
    w = np.array([cenario()['w_func'](ti) for ti in t]).T
    
    # Gera as referências x_ref(t) ao longo do tempo, shape (n, len(t))
    x_ref = np.array([cenario()['x_ref_func'](ti) for ti in t]).T
    
    # Obtém o título do cenário para os plots
    
    # Condição inicial do estado estendido: vetor z0 com zeros (para o observador), tamanho n - m
    z0 = np.zeros(n - m)
    
    # Combina o estado inicial x0 com o estado inicial do observador z0
    x0_ext = np.concatenate((x0, z0))

    # Inicializa o vetor de controle u(t) com zeros, 2 linhas (para dois atuadores), len(t) colunas
    u = np.zeros((2, len(t)))

    # Calcula o custo terminal η0 = P_LQ * x_ref no instante final
    η0 = P_LQ @ x_ref[:, -1].reshape(-1, 1)

    # Define o sistema auxiliar de tempo reverso com: -A_LQ, -Q, identidade como saída, zeros como entrada direta
    η_ss = ct.ss(-A_LQ, -Q_LQ, np.eye(6), np.zeros((6, 6)))

    # Resolve a resposta forçada do sistema auxiliar com tempo reverso
    ts_rev, η = ct.forced_response(η_ss, T=ts[::-1], U=x_ref[:, ::-1], X0=η0)
    
    # Reverte η para frente no tempo
    η = η[:, ::-1]

    # Prealoca matriz para armazenar a simulação: estados reais (n) + estados do observador (n - m)
    x_sim = np.zeros((n + n - m, len(t)))
    
    # Define a condição inicial estendida no primeiro instante
    x_sim[:, 0] = x0_ext

    # Passo de integração (assumindo espaçamento uniforme em t)
    dt = t[1] - t[0]

    # Loop de simulação ao longo do tempo
    for k in range(len(t) - 1):
        # Estado atual estendido
        xk = x_sim[:, k]
        
        # Estado real do sistema
        x_real = xk[0:n]
        
        # Estado do observador
        z_obs = xk[n:]

        # Estimativa do estado: combinação linear do estado real e do estado do observador
        x_hat = np.array(S).astype(np.float64) @ np.array(C_num).astype(np.float64) @ x_real + np.array(N_ff).astype(np.float64) @ z_obs

        # Controle por pré-alimentação: calcula v_k = R⁻¹ * B^T * η
        v_k = np.linalg.inv(R) @ B_num.T @ η[:, k]
        
        # Controle total: feedback (K_LQ * x_hat) + pré-alimentação (v_k)
        u[:, k] = -K_LQ @ x_hat + v_k

        # Dinâmica do sistema: dx = A * x + B * u + E * w
        dx = np.array(A_num).astype(np.float64) @ x_real + np.array(B_num).astype(np.float64) @ u[:, k] + np.array(E_num).astype(np.float64) @ w[:, k]
        
        # Dinâmica do observador: dz = (A22 - J*A12)*z + (A21 - J*A11)*x + (B2 - J*B1)*u
        dz = (np.array(A22).astype(np.float64) - J @ np.array(A12).astype(np.float64)) @ z_obs + \
             (np.array(A21).astype(np.float64) - J @ np.array(A11).astype(np.float64)) @ x_real[0:3] + \
             (np.array(B2).astype(np.float64) - J @ np.array(B1).astype(np.float64)) @ u[:, k]

        # Integração de x: estado real no próximo passo
        x_sim[0:n, k+1] = x_real + dx * dt
        
        # Integração de z: estado do observador no próximo passo
        x_sim[n:, k+1] = z_obs + dz * dt

    # Plotagem dos resultados: estados reais, estimados e de referência
    plt.figure(figsize=(12, 6))
    
    # Estimativa do estado ao longo de todo o tempo
    x_hat_t = np.array(S).astype(np.float64) @ np.array(C_num).astype(np.float64) @ x_sim[0:n, :] + np.array(N).astype(np.float64) @ x_sim[n:, :]
    
    # Loop para plotar cada componente do estado
    for i in range(n):
        # Estado real
        plt.plot(t, x_sim[i, :], label=f'$x_{{{i+1}}}$ real')
        # Estado estimado
        plt.plot(t, x_hat_t[i, :], '--', label=f'$\\hat{{x}}_{{{i+1}}}$ estimado')
        # Estado de referência
        plt.plot(t, x_ref[i, :], ':', label=f'$x_{{{i+1}}}^{{ref}}$')

    # Configurações do gráfico de estados
    plt.xlabel('Tempo [s]')
    plt.ylabel('Estados [rad / rad/s]')
    plt.title(title)
    plt.legend(ncol=2)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Plot do controle aplicado ao sistema
    plt.figure()
    
    # Controle do atuador direito
    plt.plot(t, u[0, :], label=r'$V_D$')
    # Controle do atuador esquerdo
    plt.plot(t, u[1, :], label=r'$V_E$')
    
    # Configurações do gráfico de controle
    plt.xlabel('Tempo [s]')
    plt.ylabel('Controle (V)')
    plt.legend()
    plt.grid(True)
    plt.title(f'Controle aplicado - {title}')
    plt.show()

In [ ]:
simular_cenario_com_pre_alimentacao(cenario1, T=10, title = 'Cenario 1')

In [ ]:
simular_cenario_com_pre_alimentacao(cenario2, T=60, title = 'Cenario 2')